In [ ]:
import pandas as pd
import numpy as np
import random
#from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Load dataset
df=pd.read_csv('SCOA_A4.csv')
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y=df['species']

# --- Genetic Algorithm Setup ---
POP_SIZE = 20      # number of individuals
N_GENERATIONS = 10 # iterations
MUTATION_RATE = 0.2

# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]

def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()

def selection(population, fitnesses):
    idx = np.argsort(fitnesses)[-2:]  # select best two
    return [population[idx[0]], population[idx[1]]]

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome

# --- Run GA ---
population = [create_chromosome() for _ in range(POP_SIZE)]

for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population

# Best result
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
print("Best Hyperparameters:", population[best_idx])
print("Best Accuracy:", fitnesses[best_idx])


Generation 0 - Best Fitness: 0.9667
Generation 1 - Best Fitness: 0.9667
Generation 2 - Best Fitness: 0.9667
Generation 3 - Best Fitness: 0.9667
Generation 4 - Best Fitness: 0.9667
Generation 5 - Best Fitness: 0.9667
Generation 6 - Best Fitness: 0.9667
Generation 7 - Best Fitness: 0.9667
Generation 8 - Best Fitness: 0.9733
Generation 9 - Best Fitness: 0.9733
Best Hyperparameters: [3, 4]
Best Accuracy: 0.9733333333333334


In [ ]:
'''
import pandas as pd
import numpy as np
import random
#from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
```

* `pandas` (pd): used to read and handle table data (CSV).
* `numpy` (np): used for numeric helpers (e.g. argsort).
* `random`: Python’s simple random-number utilities (used for GA ops).
* `cross_val_score`: evaluates a model using cross-validation (gives accuracy estimates).
* `DecisionTreeClassifier`: the ML model whose hyperparameters we tune.
* `#from sklearn.datasets import load_iris` is commented out — not used here.

---

```python
# Load dataset
df=pd.read_csv('SCOA_A4.csv')
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y=df['species']
```

* Read the CSV file `'SCOA_A4.csv'` into a DataFrame `df`.
* `X` is a table of input features (4 numeric columns — typical iris features).
* `y` is the target column (species labels to predict).

---

```python
# --- Genetic Algorithm Setup ---
POP_SIZE = 20      # number of individuals
N_GENERATIONS = 10 # iterations
MUTATION_RATE = 0.2
```

* `POP_SIZE`: how many candidate solutions ("individuals") exist in each generation.
* `N_GENERATIONS`: how many times the population evolves (how many iterations).
* `MUTATION_RATE`: probability (20%) to randomly change a gene when mutating.

---

```python
# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]
```

* A **chromosome** encodes a candidate set of hyperparameters for the decision tree.
* Here a chromosome is a list of two integers:

  * `max_depth` (how deep the tree can grow) between 1 and 20.
  * `min_samples_split` (minimum samples to split a node) between 2 and 10.
* `create_chromosome()` randomly initializes one such chromosome.

---

```python
def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()
```

* `fitness` measures how good a chromosome is.
* It unpacks the two genes into `max_depth` and `min_samples_split`.
* Builds a `DecisionTreeClassifier` with those hyperparameters.
* Evaluates the model using 5-fold cross validation on (X,y).
* Returns the **mean cross-validation score** (average accuracy across folds).
* Higher fitness = better hyperparameters.

---

```python
def selection(population, fitnesses):
    idx = np.argsort(fitnesses)[-2:]  # select best two
    return [population[idx[0]], population[idx[1]]]
```

* `selection` chooses parents from the population for reproduction.
* `np.argsort(fitnesses)` returns indices that would sort fitnesses ascending.
* `[-2:]` picks indices of the two best individuals (highest fitness).
* Then it returns the corresponding chromosomes as the selected parents.
* (Note: this always selects the top 2 — no tournament or roulette wheel here.)

---

```python
def crossover(parent1, parent2):
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2
```

* `crossover` mixes genes of two parents to create two children.
* `point` is a random crossover index (0..number_of_genes-1).
* `child1` takes genes from `parent1` up to `point`, then from `parent2` after `point`.
* `child2` does the opposite.
* This is **single-point crossover** on a short list of two genes.

---

```python
def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome
```

* `mutate` randomly changes genes in a chromosome with probability `MUTATION_RATE`.
* If a mutation occurs on a gene, it assigns a new random integer within the allowed range.
* Mutation injects genetic diversity and helps escape local optima.

---

```python
# --- Run GA ---
population = [create_chromosome() for _ in range(POP_SIZE)]
```

* Initializes the starting population: `POP_SIZE` randomly created chromosomes.

---

```python
for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population
```

* Loop for `N_GENERATIONS` rounds:

  1. Compute fitness for each chromosome in the population.
  2. Print the best fitness of this generation (for monitoring).
  3. Select the top two parents using `selection`.
  4. Produce `POP_SIZE` children by repeatedly crossing the same two parents.

     * Each loop iteration creates two children, so loop runs `POP_SIZE // 2` times.
     * After crossover, each child is mutated and appended to `new_population`.
  5. Replace the old population with the newly produced population.
* **Important behavior**: This GA uses only the top two parents to generate the entire next population. (No parent diversity beyond those two.)

---

```python
# Best result
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
print("Best Hyperparameters:", population[best_idx])
print("Best Accuracy:", fitnesses[best_idx])
```

* After the final generation, compute fitnesses once more.
* Find the index of the best chromosome (`np.argmax`).
* Print the best hyperparameters and their fitness (accuracy).

---

# Short plain-English summary (what the program does)

1. Read a dataset and separate features (`X`) and labels (`y`).
2. Create a population of random candidate hyperparameter settings for a decision tree (each candidate = `[max_depth, min_samples_split]`).
3. For a fixed number of generations:

   * Evaluate each candidate by training a decision tree and computing mean 5-fold CV accuracy (that is its fitness).
   * Choose the two best candidates as parents.
   * Repeatedly cross those two parents to create new children, mutate the children slightly, and form the next population.
4. After evolution ends, report the best hyperparameters found and their cross-validation accuracy.

# A few practical notes / suggestions

* **Selection is greedy**: you always pick the best two parents only. That can make the GA converge quickly but risks losing diversity (and getting stuck). Consider selecting more parents or using tournament/roulette selection.
* **Crossover on 2-gene chromosomes**: with two genes, crossover only swaps genes in a limited way — fine for this small search space, but you may want to use more sophisticated encodings if you add more hyperparameters.
* **Population diversity**: because only two parents produce the entire next generation, population diversity is low. You could keep some elites and also allow random parents to mate.
* **Fitness cost**: `cross_val_score` trains/evaluates a model many times — this is expensive. For small datasets it’s fine; for large data consider fewer folds or other validation strategies.
* **Reproducibility**: you might want to seed `random` and `numpy` to make runs reproducible (`random.seed(0)` and `np.random.seed(0)`).
* **Hyperparameter ranges**: ranges (1–20, 2–10) are arbitrary; tune them to suit your problem.
* **Stopping criteria**: instead of fixed `N_GENERATIONS`, you can stop if fitness doesn’t improve for many generations.

'''

'''Excellent question, Saniya 👏 — cross-validation (especially **k-fold cross-validation**) is a *very important concept* in machine learning. Let’s go step-by-step in **simple words** with an example.

---

### 🎯 **What is Cross-Validation?**

Cross-validation is a **technique used to test how well a machine learning model will perform on unseen data**.

👉 In short, it helps us avoid **overfitting** (when a model performs well on training data but fails on new data).

---

### 💡 **Why do we need it?**

If we only train and test our model on the same data, it might **memorize** patterns instead of learning them.
So we need a smarter way to check:

> “Can my model generalize well to data it has never seen?”

That’s where **cross-validation** comes in.

---

### ⚙️ **How does it work (step-by-step)?**

Let’s take an example with **k = 5 folds** — i.e., **5-fold cross-validation**.

Suppose you have **100 data samples**.

1. **Split the data into 5 equal parts** (each with 20 samples).

   ```
   Fold1, Fold2, Fold3, Fold4, Fold5
   ```

2. **Train and test multiple times:**

   * Round 1 → Train on Fold2–Fold5, Test on Fold1
   * Round 2 → Train on Fold1, Fold3–Fold5, Test on Fold2
   * Round 3 → Train on Fold1–Fold2, Fold4–Fold5, Test on Fold3
   * Round 4 → Train on Fold1–Fold3, Fold5, Test on Fold4
   * Round 5 → Train on Fold1–Fold4, Test on Fold5

3. Each round gives one **accuracy score**.

4. Finally, take the **average of all 5 accuracies** — this gives a more **reliable estimate** of model performance.

---

### 🧠 **In your code:**

You have this line:

```python
scores = cross_val_score(model, X, y, cv=5)
```

* `model`: your DecisionTreeClassifier
* `X`: input features
* `y`: target labels
* `cv=5`: perform **5-fold cross-validation**

👉 What happens here:

* Your dataset is split into 5 parts.
* The model trains on 4 parts and tests on the remaining one.
* This repeats 5 times (so each part gets to be the test set once).
* The function returns 5 accuracy scores (one per round).

Then you do:

```python
return scores.mean()
```

That means you’re taking the **average accuracy** over all 5 folds.
This average is used as the **fitness score** of that model configuration in your genetic algorithm.

---

### 📊 **Analogy (in simple words):**

Imagine you’re preparing for an exam.
Instead of testing yourself on the same questions again and again (which you might memorize),
you split the syllabus into 5 parts and test yourself on each part separately.
Your average performance gives a **fair measure** of how prepared you are overall.

---

### ✅ **Benefits of Cross-Validation**

* Gives a **more accurate** estimate of model performance.
* Prevents **overfitting**.
* Works well when the dataset is **small**.
* Ensures every data point is used for both **training and testing** at least once.

---

Would you like me to show this concept visually using a **diagram** (like how data folds are rotated across rounds)? It’ll make it crystal clear.
'''